<a href="https://colab.research.google.com/github/fernandodeeke/epidemias/blob/main/seirs_saz_intervalos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
import time
from scipy.signal import find_peaks

# Função para definir h(t) com base no calendário escolar brasileiro
def h_tempo(t):
    t = t % 365  # Considera o ano cíclico
    if 0 <= t <= 45:  # 1 jan - 15 fev (férias de verão)
        return 0
    elif 46 <= t <= 180:  # 16 fev - 30 jun (aulas)
        return 1
    elif 181 <= t <= 195:  # 1 jul - 15 jul (férias de julho)
        return 0
    elif 196 <= t <= 348:  # 16 jul - 15 dez (aulas)
        return 1
    else:  # 16 dez - 31 dez (férias de verão)
        return 0

# Função que simula o modelo SEIRS com transmissão por frequência
def simula_seirs_sazonal(A, epsilon, nu, mu, alpha, sigma, gamma, omega, days, S0, E0, I0, N0):
    passos = days + 1
    tempo = np.arange(0, passos)

    # Arrays para os compartimentos
    S = np.zeros(passos)
    E = np.zeros(passos)
    I = np.zeros(passos)
    R = np.zeros(passos)
    N = np.zeros(passos)

    # Condições iniciais
    N[0] = N0
    S[0] = S0
    E[0] = E0
    I[0] = I0
    R[0] = max(N0 - S0 - E0 - I0, 0)

    # Simulação com equações de diferença
    for n in range(passos - 1):
        beta_n = A * (1 + epsilon * h_tempo(n))
        if N[n] > 0:  # Evita divisão por zero
            dS = nu * N[n] - beta_n * S[n] * I[n] / N[n] - mu * S[n] + omega * R[n]
            dE = beta_n * S[n] * I[n] / N[n] - (sigma + mu) * E[n]
        else:
            dS = 0
            dE = 0
        dI = sigma * E[n] - (gamma + mu + alpha) * I[n]
        dR = gamma * I[n] - (mu + omega) * R[n]

        S[n+1] = max(S[n] + dS, 0)
        E[n+1] = max(E[n] + dE, 0)
        I[n+1] = max(I[n] + dI, 0)
        R[n+1] = max(R[n] + dR, 0)
        N[n+1] = S[n+1] + E[n+1] + I[n+1] + R[n+1]

    return tempo, I, N

# Interface interativa com sliders
def interactive_seirs_sazonal():
    sliders = {
        "A": FloatSlider(value=0.25, min=0.0, max=1.0, step=0.01, description='A (β)'),
        "epsilon": FloatSlider(value=0.4, min=0.0, max=1.0, step=0.01, description='ε'),
        "nu": FloatSlider(value=0.000038, min=0.0, max=0.001, step=0.00001, description='ν', readout_format='.6f'),
        "mu": FloatSlider(value=0.000022, min=0.0, max=0.001, step=0.00001, description='μ', readout_format='.6f'),
        "alpha": FloatSlider(value=0.00005, min=0.0, max=0.001, step=0.00001, description='α', readout_format='.6f'),
        "sigma": FloatSlider(value=1/4, min=0.1, max=1.0, step=0.01, description='σ', readout_format='.3f'),
        "gamma": FloatSlider(value=1/5, min=0.01, max=1.0, step=0.01, description='γ', readout_format='.3f'),
        "omega": FloatSlider(value=1/365, min=0.0, max=0.01, step=0.0001, description='ω', readout_format='.4f'),
        "days": IntSlider(value=365*6, min=30, max=10000, step=1, description='Dias'),
        "S0": IntSlider(value=95000, min=0, max=100000, step=100, description='S0'),
        "E0": IntSlider(value=100, min=0, max=1000, step=10, description='E0'),
        "I0": IntSlider(value=50, min=0, max=1000, step=10, description='I0'),
        "N0": IntSlider(value=100000, min=1000, max=200000, step=1000, description='N0')
    }

    def update_plot(**kwargs):
        start_time = time.process_time()
        tempo, I, N = simula_seirs_sazonal(**kwargs)
        end_time = time.process_time()
        print(f"Tempo de CPU: {end_time - start_time:.4f} segundos")

        tempo_anos = tempo / 365.0  # Converte para anos
        I_percent = (I / N) * 100  # Proporção I(t)/N em porcentagem

        # Encontrar os picos relevantes de I(t)/N
        picos, properties = find_peaks(I_percent,
                                       height=0.5,       # Altura mínima de 0.5%
                                       distance=90,      # Distância mínima de 90 dias entre picos
                                       prominence=0.1)   # Proeminência mínima de 0.1%
        dias_picos = tempo[picos]  # Dias dos picos
        valores_picos = I_percent[picos]  # Valores dos picos em %

        # Calcular os intervalos interepidêmicos
        if len(dias_picos) > 1:
            intervalos = np.diff(dias_picos)  # Diferenças entre dias consecutivos
            intervalos_anos = intervalos / 365.0  # Converte para anos
            print("\nPicos relevantes de I(t)/N (%):")
            for i, (dia, valor) in enumerate(zip(dias_picos, valores_picos)):
                print(f"Pico {i+1}: Dia {dia} ({dia/365:.2f} anos), I/N = {valor:.2f}%")
            print("\nIntervalos interepidêmicos (em dias e anos):")
            for i, (inter_dias, inter_anos) in enumerate(zip(intervalos, intervalos_anos)):
                print(f"Intervalo {i+1}: {inter_dias} dias ({inter_anos:.2f} anos)")
        else:
            print("Apenas um pico ou nenhum pico identificado com os critérios fornecidos.")

        # Gráfico
        plt.figure(figsize=(10, 6))
        plt.plot(tempo_anos, I_percent, lw=2, color='red')  # Apenas I(t)/N em %
        plt.plot(tempo_anos[picos], I_percent[picos], "x", color='black')  # Marca os picos
        plt.xlabel("Anos")
        plt.ylabel("Proporção de Infectados (%)")
        plt.grid()
        plt.show()

    interact(update_plot, **sliders)

# Executa a simulação interativa
interactive_seirs_sazonal()

interactive(children=(FloatSlider(value=0.25, description='A (β)', max=1.0, step=0.01), FloatSlider(value=0.4,…